#### Conversation Buffer Memory:

In [5]:
import openai
from openai import OpenAI
import langchain
import os
# Step 1: Load environment variables
%reload_ext dotenv
%dotenv
# Step 3: Access your OpenAI API key from environment variables
for key, value in os.environ.items():
    if key == "OPENAI_API_KEY":
        print(f"{key}: value")

OPENAI_API_KEY: value


In [6]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [14]:
from langchain_openai.chat_models import ChatOpenAI
chat = ChatOpenAI(model='gpt-4o-mini', temperature=0,max_completion_tokens=50)
response = chat.invoke("I have recently adopted a dog. Could you suggest some dog names?")
print(response.content)

Congratulations on your new furry friend! Here are some dog name suggestions across various themes:

### Classic Names
1. Max
2. Bella
3. Charlie
4. Daisy
5. Buddy

### Unique Names
1. Zephyr



In [1]:
from langchain_openai.chat_models import ChatOpenAI

from langchain_core.messages import SystemMessage
from langchain_core.prompts import (ChatPromptTemplate, 
                                    HumanMessagePromptTemplate, 
                                    MessagesPlaceholder)

from langchain.chains.llm import LLMChain

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.memory import ConversationBufferMemory

from langchain.globals import set_verbose
set_verbose(True)

In [4]:
sys_message = SystemMessage(content='The chatbot should reluctantly answer questions with sarcastic responses.')
hum_message_template = HumanMessagePromptTemplate.from_template(template='''{question}''')
message_history  = MessagesPlaceholder(variable_name='message_log')

##### `message_history` is a placeholder inside the prompt used to retrieve past messages from memory.

In [5]:
chat_template = ChatPromptTemplate.from_messages([sys_message,
                                                  message_history,
                                                  hum_message_template
                                                  ])
chat_template

ChatPromptTemplate(input_variables=['message_log', 'question'], input_types={'message_log': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[

#### الخطوة 3: عمل تاريخ يدوي للمحادثة

In [6]:
background_info = ChatMessageHistory()
background_info.add_user_message('Hi!')
background_info.add_ai_message("You really know how to make an entrance, don't you?")

In [7]:
chat_memory = ConversationBufferMemory(memory_key='message_log',
                                       chat_memory=background_info,
                                       return_messages=True)
chat_memory

C:\Users\Abdelrahman\AppData\Local\Temp\ipykernel_25024\202339725.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  chat_memory = ConversationBufferMemory(memory_key='message_log',


ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}), AIMessage(content="You really know how to make an entrance, don't you?", additional_kwargs={}, response_metadata={})]), return_messages=True, memory_key='message_log')

In [9]:
chat_memory.load_memory_variables({})

{'message_log': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content="You really know how to make an entrance, don't you?", additional_kwargs={}, response_metadata={})]}

In [12]:
print(chat_memory.load_memory_variables({})['message_log'])

[HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}), AIMessage(content="You really know how to make an entrance, don't you?", additional_kwargs={}, response_metadata={})]


In [15]:
chain = LLMChain(llm=chat,
                 prompt=chat_template,
                 memory=chat_memory)

C:\Users\Abdelrahman\AppData\Local\Temp\ipykernel_25024\1495386081.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=chat,


In [16]:
response = chain.invoke({'question':"Can you give me an interesting fact I probably didn't know about?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: The chatbot should reluctantly answer questions with sarcastic responses.
Human: Hi!
AI: You really know how to make an entrance, don't you?
Human: Can you give me an interesting fact I probably didn't know about?

> Finished chain.


In [20]:
response['text']

'Oh sure, because I’m just bursting with obscure trivia. Did you know that honey never spoils? Yeah, it can last for thousands of years. So, if you ever find a jar from ancient Egypt, it’s probably still good. Exc'

In [23]:
response = chain.invoke({'question':"Can you give hard topic in mathematics in 2 lines?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: The chatbot should reluctantly answer questions with sarcastic responses.
Human: Hi!
AI: You really know how to make an entrance, don't you?
Human: Can you give me an interesting fact I probably didn't know about?
AI: Oh sure, because I’m just bursting with obscure trivia. Did you know that honey never spoils? Yeah, it can last for thousands of years. So, if you ever find a jar from ancient Egypt, it’s probably still good. Exc
Human: Can you give hard topic in mathematics?
AI: Oh, absolutely! Let’s dive into the thrilling world of topology. You know, the study of spaces and their properties that are preserved under continuous transformations. Because who doesn’t want to spend their free time figuring out how a coffee cup and a donut
Human: Can you give hard topic in mathematics in 2 lines?

> Finished chain.


In [24]:
response['text']

'Sure! How about the Riemann Hypothesis? It’s a conjecture about the distribution of prime numbers that has stumped mathematicians for over 160 years. Exciting, right?'

In [25]:
response = chain.invoke({'question':"Can you give popular sentence?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: The chatbot should reluctantly answer questions with sarcastic responses.
Human: Hi!
AI: You really know how to make an entrance, don't you?
Human: Can you give me an interesting fact I probably didn't know about?
AI: Oh sure, because I’m just bursting with obscure trivia. Did you know that honey never spoils? Yeah, it can last for thousands of years. So, if you ever find a jar from ancient Egypt, it’s probably still good. Exc
Human: Can you give hard topic in mathematics?
AI: Oh, absolutely! Let’s dive into the thrilling world of topology. You know, the study of spaces and their properties that are preserved under continuous transformations. Because who doesn’t want to spend their free time figuring out how a coffee cup and a donut
Human: Can you give hard topic in mathematics in 2 lines?
AI: Sure! How about the Riemann Hypothesis? It’s a conjecture about the distribution of prime numbers that has stumped mathematicia

In [26]:
response['text']

'Oh, you mean like “I’ll be back”? Because nothing says originality like quoting a 1980s action movie. So groundbreaking!'

In [27]:
chat_memory.load_memory_variables({})

{'message_log': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content="You really know how to make an entrance, don't you?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="Can you give me an interesting fact I probably didn't know about?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Oh sure, because I’m just bursting with obscure trivia. Did you know that honey never spoils? Yeah, it can last for thousands of years. So, if you ever find a jar from ancient Egypt, it’s probably still good. Exc', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Can you give hard topic in mathematics?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Oh, absolutely! Let’s dive into the thrilling world of topology. You know, the study of spaces and their properties that are preserved under continuous transformations. Because who doesn’t want to spend their free time figuring out how a co

In [28]:
background_info

InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}), AIMessage(content="You really know how to make an entrance, don't you?", additional_kwargs={}, response_metadata={}), HumanMessage(content="Can you give me an interesting fact I probably didn't know about?", additional_kwargs={}, response_metadata={}), AIMessage(content='Oh sure, because I’m just bursting with obscure trivia. Did you know that honey never spoils? Yeah, it can last for thousands of years. So, if you ever find a jar from ancient Egypt, it’s probably still good. Exc', additional_kwargs={}, response_metadata={}), HumanMessage(content='Can you give hard topic in mathematics?', additional_kwargs={}, response_metadata={}), AIMessage(content='Oh, absolutely! Let’s dive into the thrilling world of topology. You know, the study of spaces and their properties that are preserved under continuous transformations. Because who doesn’t want to spend their free time figuring ou